In [1]:
from examples.promoters.data import Promoters
from torch.utils.data import Dataset, DataLoader
import torch

dataset = Promoters([
        "/home/slavic/PycharmProjects/promoters16/fold_1.csv",
        # "/home/slavic/PycharmProjects/promoters16/fold_2.csv",
        # "/home/slavic/PycharmProjects/promoters16/fold_3.csv",
        # "/home/slavic/PycharmProjects/promoters16/fold_4.csv",
        # "/home/slavic/PycharmProjects/promoters16/fold_5.csv",
])

train_data, test_data = torch.utils.data.random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])

train_loader = DataLoader(train_data, shuffle=True, batch_size=32)
test_loader = DataLoader(test_data, shuffle=False, batch_size=256)

/home/slavic/PycharmProjects/promoters16/fold_1.csv


In [2]:
from examples.promoters.modules import DataType
from data_filters.sliding_window import SlidingWindowFilterTuple

rollout = 1000
data_filter = SlidingWindowFilterTuple[DataType](rollout, padding=200, skip_fields={"target", "length"})

In [3]:
from memup.preproc import IncrementStep
from examples.promoters.modules import MemUpMemoryImpl
from common_modules.transformers import BertRecurrentTransformerWithTokenizer
from transformers import AutoTokenizer
from gena_lm.modeling_bert import BertModel, BertForSequenceClassification
from memup.base import MemoryRollout

tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-base')
bert_model: BertModel = BertForSequenceClassification.from_pretrained('AIRI-Institute/gena-lm-bert-base').bert
mem_transformer = BertRecurrentTransformerWithTokenizer(bert_model, tokenizer, 300, 4, 3, bert_model.config.hidden_size * 2).cuda()

memup_iter = MemoryRollout[DataType](
    steps=2,
    memory=MemUpMemoryImpl(mem_transformer),
    data_filter=data_filter,
    info_update=[IncrementStep()]
)

Some weights of the model checkpoint at AIRI-Institute/gena-lm-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [4]:
from torch import Tensor
from memup.loss import TS
from memup.base import MemoryOut, State, DataCollectorReplace


class DataCollectorTrain(DataCollectorReplace[DataType, Tensor]):
    def apply(self, data: DataType, out: MemoryOut, state: State) -> Tensor:
        return state

In [5]:
for text, labels in train_loader:
        print()

        state = torch.zeros(labels.shape[0], 50, bert_model.config.hidden_size).cuda()
        T = len(text[0])
        done = False
        info = {}


        while not done:
                data_collector, state, info, done = memup_iter.forward(DataType(text, labels, T), state, info, DataCollectorTrain())
                s = data_collector.result()
                print(s[-1].shape)


/home/slavic/PycharmProjects/venv/lib/python3.10/site-packages/transformers/modeling_utils.py:768: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])

torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])

torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])

torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])

torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Size([32, 50, 768])
torch.Si

KeyboardInterrupt: 